# This notebook makes the Pseudobulks from processed single-cell data and formats relevant metadata

Data format requirements for single-cell data:
- processed data is not scaled
- cells are filtered such that low-quality cells are removed (for example: filter out cells with less than 200 genes and genes expressed in less than 3 cells, and > 5% MT reads)
- data is saved as an AnnData object and you have sample IDs, gene IDs, and cell-type labels

The end product of this notebook will include the following `.pkl` files per each sample, stimulation and datasplit combination:
- Pseudobulk gene expression (`*_pseudo_splits.pkl`) and proportions (`*_prop_splits.pkl`) from random cell type mixture and single cell type dominant mixture
- Metadata (`*_meta_splits.pkl`) with the following fields
  - the observations have columns named: "sample_id", "stim", "isTraining"
  - sample_id: unique IDs for the samples
  - stim: is "STIM" or "CTRL", denotes if the sample is "female" or "male"
  - isTraining: 'Train' or 'Test',  denotes whether the sample is used during training or not.

In [1]:
import sys
import pathlib
import yaml
import subprocess
import pickle

import pandas as pd
import numpy as np
import scanpy as sc
import anndata as ad
from sklearn.model_selection import train_test_split

## Preprocessing Parameters

In [2]:
CELL_TYPE_COL = 'encode_celltype'
SAMPLE_ID_COL = 'sample_id'
STIM_COL = 'stim'

GENE_ID_COL = 'gene_ids'

DATASPLIT_COL = 'isTraining'

DATASPLIT_SEED = 42

## Load config
The config file specifies the path to data and software repo (due to currently in active development)

In [3]:
# Get the root directory of the analysis repository
REPO_ROOT = subprocess.run(
    ["git", "rev-parse", "--show-toplevel"], capture_output=True, text=True
).stdout.strip()
REPO_ROOT = pathlib.Path(REPO_ROOT)

CONFIG_FILE = REPO_ROOT / 'config.yml'
assert CONFIG_FILE.exists(), f"Config file not found at {CONFIG_FILE}"

with open(CONFIG_FILE, 'r') as file:
    config_dict = yaml.safe_load(file)

## Add dev buddi fork to path and import

In [4]:
buddi_fork_path = config_dict['software_path']['buddi_HGSC']
buddi_fork_path = pathlib.Path(buddi_fork_path)
assert buddi_fork_path.exists(), f"buddi fork not found at {buddi_fork_path}"

sys.path.insert(0, str(buddi_fork_path))
# this is quite ugly, once activate modifications are done this will be changed
# to a proper installation + import
from buddi import preprocessing
from buddi.preprocessing import utils
from buddi.preprocessing import generate_pseudo_bulks

## Retrieve Path to Processed Single-Cell RNA-seq Data and relevant Metadata

In [5]:
STUDY_GEO_ID = 'GSE154600' # TODO consider whether to move this into config.yml as well
SC_DATA_PATH = pathlib.Path(config_dict['data_path']['sc_data_path'])

SC_ADATA_PATH = SC_DATA_PATH / f'{STUDY_GEO_ID}_processed'
assert SC_ADATA_PATH.exists(), f"Processed Single-cell Data path {SC_ADATA_PATH} does not exist"
SC_ADATA_FILE = SC_ADATA_PATH / f'{STUDY_GEO_ID}_processed.h5ad'
assert SC_ADATA_FILE.exists(), f"Processed Single-cell Data file {SC_ADATA_FILE} does not exist"

SC_METADATA_PATH = SC_DATA_PATH / f'{STUDY_GEO_ID}_metadata'
assert SC_METADATA_PATH.exists(), f"Single-cell Metadata path {SC_METADATA_PATH} does not exist"

## Define Path to write Pre-Processing Outputs

In [6]:
PREPROCESSING_OUTPUT_PATH = REPO_ROOT / 'processed_data'
assert PREPROCESSING_OUTPUT_PATH.exists(), f"Preprocessing output path {PREPROCESSING_OUTPUT_PATH} does not exist"
SC_AUGMENTED_DATA_PATH = PREPROCESSING_OUTPUT_PATH / 'sc_augmented'
SC_AUGMENTED_DATA_PATH.mkdir(exist_ok=True, parents=True)

## Preprocessing of scRNA-seq Anndata before Moving to Pseudobulk
### Load and Preprocess Anndata

In [7]:
adata = sc.read_h5ad(SC_ADATA_FILE)

# checking if the defined columns are present in the adata.obs
assert CELL_TYPE_COL in adata.obs.columns, f"Column {CELL_TYPE_COL} not found in adata.obs"
assert SAMPLE_ID_COL in adata.obs.columns, f"Column {SAMPLE_ID_COL} not found in adata.obs"
assert STIM_COL in adata.obs.columns, f"Column {STIM_COL} not found in adata.obs"

In [8]:
adata.var_names_make_unique()
adata.var[GENE_ID_COL] = adata.var.index.tolist()

# replace underscores with hyphens in the sample_id column
adata.obs[SAMPLE_ID_COL] = adata.obs[SAMPLE_ID_COL].str.replace('_', '-')

### Print some basic information

In [9]:
print(adata.shape)
print(adata.var.head())
print(adata.obs.head())

(36111, 24520)
              gene_ids  n_cells     mt  n_cells_by_counts  mean_counts  \
gene_ids                                                                 
AL627309.1  AL627309.1       23  False                 23     0.000614   
AL669831.5  AL669831.5      629  False                629     0.017388   
FAM87B          FAM87B       26  False                 26     0.000694   
LINC00115    LINC00115      463  False                463     0.012661   
FAM41C          FAM41C      279  False                279     0.007746   

            pct_dropout_by_counts  total_counts  
gene_ids                                         
AL627309.1              99.938567          23.0  
AL669831.5              98.319934         651.0  
FAM87B                  99.930554          26.0  
LINC00115               98.763322         474.0  
FAM41C                  99.254788         290.0  
                           GSM             Barcode  Cluster          cellType  \
AAACCTGAGCTGCCCA-1  GSM4675273  AAA

## Some Stats

In [10]:
tab = adata.obs.groupby([SAMPLE_ID_COL, STIM_COL]).size()
tab.unstack()

stim,CTRL
sample_id,
Samp-T59,11689
Samp-T76,11876
Samp-T77,4974
Samp-T89,4291
Samp-T90,3281


In [11]:
adata.obs[CELL_TYPE_COL].value_counts()

CD8+ T-cells         8183
Fibroblasts          4385
Epithelial cells     4021
Macrophages          3979
Adipocytes           3955
Monocytes            3507
Mesangial cells      3488
CD4+ T-cells         1476
NK cells             1409
B-cells              1169
Endothelial cells     539
Name: encode_celltype, dtype: int64

## Output Gene ids

In [13]:
gene_out_file = SC_AUGMENTED_DATA_PATH / f'{STUDY_GEO_ID}_genes.pkl'
gene_ids = adata.var[GENE_ID_COL]
pickle.dump(gene_ids, open( gene_out_file, "wb" ) )

## Make Pseudobulks
### First perform random train test split stratifying by sample id and stimulation

In [14]:
# Split the data into train and test sets
train_idx, test_idx = train_test_split(
    adata.obs.index,
    test_size=0.2,
    stratify=adata.obs[[SAMPLE_ID_COL, STIM_COL, CELL_TYPE_COL]],
    random_state=DATASPLIT_SEED
)

# Assign the split to the DATASPLIT_COL
adata.obs.loc[train_idx, DATASPLIT_COL] = 'Train'
adata.obs.loc[test_idx, DATASPLIT_COL] = 'Test'

In [15]:
ADD_PER_CELL_TYPE_NOISE = True
N_CELLS_PER_PSEUDO_BULK = 5_000
N_PSEUDO_BULKS_PER_CONDITION = 1_000

gene_ids = adata.var[GENE_ID_COL]
samples = adata.obs[SAMPLE_ID_COL].unique()
stims = adata.obs[STIM_COL].unique()

## Global cell types
cell_types = adata.obs[CELL_TYPE_COL].unique()
cell_order = sorted(cell_types) # format the proportion columns in the same order
datasplits = adata.obs[DATASPLIT_COL].unique()

n_samples = len(samples)
n_genes = len(gene_ids)
n_cell_types = len(cell_order)

# Define cell-type level noise for the generated pseudo-bulk profiles
if ADD_PER_CELL_TYPE_NOISE:
    # this produces a list of numpy arrays, each of length n_genes
    # to reflect the expression noise associated with each specific cell type
    per_cell_type_noise = [
        np.random.lognormal(0, 0, n_genes) for i in range(n_cell_types)]
else:
    per_cell_type_noise = None

# Generate pseudo-bulk profiles grouping by sample_id and stim
for _sample in samples:
    for _stim in stims:
        for _datasplit in datasplits:

            print(f"Generating pseudo-bulk profiles for sample {_sample}, stim {_stim}, and datasplit {_datasplit} ...")
            
            ## Subset adata to the current sample, stim and train/test split
            subset_idx = np.where(
                np.logical_and.reduce((
                    adata.obs[SAMPLE_ID_COL] == _sample, 
                    adata.obs[STIM_COL] == _stim,
                    adata.obs[DATASPLIT_COL] == _datasplit
                ))
            )[0]
            
            if len(subset_idx) == 0:
                continue
            subset_adata = adata[subset_idx, :]

            ## Cell type that is present in the subset, will inform the 
            ## down stream workflow of potential missing cell types to skip
            present_cell_types = subset_adata.obs[CELL_TYPE_COL].unique().to_list()

            ## Subset the cell_df to the present cell types
            cell_df = preprocessing.utils.subset_adata_by_cell_type(
                subset_adata, 
                cell_type_col=CELL_TYPE_COL,
                cell_order=cell_order
            )

            print("\tGenerating random prop pseudo-bulk profiles ...")

            random_count_df = preprocessing.utils.generate_log_normal_counts(
                cell_order=cell_order, 
                num_cells=N_CELLS_PER_PSEUDO_BULK, 
                num_samples=N_PSEUDO_BULKS_PER_CONDITION,
                present_cell_types=present_cell_types
            )
            random_props_df = preprocessing.utils.generate_prop_from_counts(
                random_count_df,
            )
            random_pseudobulk_df = preprocessing.generate_pseudo_bulks.generate_pseudo_bulk_from_counts(
                in_adata=subset_adata,
                cell_df=cell_df,
                count_df=random_count_df,
                cell_noise=per_cell_type_noise,
                use_sample_noise=False
            )

            print("\tGenerating single cell dominant pseudo-bulk profiles ...")

            single_cell_props_df = preprocessing.utils.generate_single_celltype_dominant_props(
                num_samp=100,
                cell_order=cell_order,
                present_cell_types=present_cell_types
            )
            single_cell_counts_df = preprocessing.utils.generate_counts_from_props(
                single_cell_props_df,
                num_cells=N_CELLS_PER_PSEUDO_BULK
            )
            single_cell_pseudobulk_df = preprocessing.generate_pseudo_bulks.generate_pseudo_bulk_from_counts(
                in_adata=subset_adata,
                cell_df=cell_df,
                count_df=single_cell_counts_df,
                cell_noise=per_cell_type_noise,
                use_sample_noise=False
            )

            print('Concatenating the two types of pseudo-bulk profiles ...')
            props_df = pd.concat([random_props_df, single_cell_props_df])
            pseudobulk_df = pd.concat([random_pseudobulk_df, single_cell_pseudobulk_df])

            ## Metadata assembly            
            n_random_prop_pbs = len(random_props_df)
            n_single_celltype_pbs = len(single_cell_pseudobulk_df)
            n_total_pbs = n_random_prop_pbs + n_single_celltype_pbs

            metadata_df = pd.DataFrame(
                data = {"sample_id":[_sample]*n_total_pbs,
                        "stim":[_stim]*n_total_pbs,
                        "isTraining":[_datasplit]*n_total_pbs,
                        "cell_prop_type":['random']*n_random_prop_pbs + ['single_celltype']*n_single_celltype_pbs,
                        "samp_type":['sc_ref']*n_total_pbs}
                        )
            
            print("Writing the pseudo-bulk profiles ...")
            pseudobulk_file = SC_AUGMENTED_DATA_PATH / f'{STUDY_GEO_ID}_{_sample}_{_stim}_{_datasplit}_pseudo_splits.pkl'
            prop_file = SC_AUGMENTED_DATA_PATH / f'{STUDY_GEO_ID}_{_sample}_{_stim}_{_datasplit}_prop_splits.pkl'
            meta_file = SC_AUGMENTED_DATA_PATH / f'{STUDY_GEO_ID}_{_sample}_{_stim}_{_datasplit}_meta_splits.pkl'

            pickle.dump( props_df, open( prop_file, "wb" ) )
            pickle.dump( pseudobulk_df, open( pseudobulk_file, "wb" ) )
            pickle.dump( metadata_df, open( meta_file, "wb" ) )

Generating pseudo-bulk profiles for sample Samp-T59, stim CTRL, and datasplit Train ...
	Generating random prop pseudo-bulk profiles ...
Processing sample 0
Processing sample 100
Processing sample 200
Processing sample 300
Processing sample 400
Processing sample 500
Processing sample 600
Processing sample 700
Processing sample 800
Processing sample 900
	Generating single cell dominant pseudo-bulk profiles ...
Processing sample 0
Processing sample 100
Processing sample 200
Processing sample 300
Processing sample 400
Processing sample 500
Processing sample 600
Processing sample 700
Processing sample 800
Processing sample 900
Processing sample 1000
Concatenating the two types of pseudo-bulk profiles ...
Writing the pseudo-bulk profiles ...
Generating pseudo-bulk profiles for sample Samp-T59, stim CTRL, and datasplit Test ...
	Generating random prop pseudo-bulk profiles ...
Processing sample 0
Processing sample 100
Processing sample 200
Processing sample 300
Processing sample 400
Processin